In [1]:
from sklearn.svm import SVR
from sklearn.model_selection import LeaveOneOut, GridSearchCV
import scipy.io
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [2]:
path_code = os.getcwd()
PA_dataset = scipy.io.loadmat(os.path.join(path_code, "../Data/PA_regr.mat"))

In [3]:
ar_m = PA_dataset["ar"]
af_m = PA_dataset["af"]
y_m = PA_dataset["slp"]


In [16]:
len(ar_m[0])

19

In [17]:
ar = np.zeros([ar_m.size[0], 1])
for i in range(ar_m.size[0]):
    ar[i,0] = ar_m[i][0][0][0]

af = np.zeros([af_m.size[0], 1])
for i in range(af_m.size[0]):
        af[i,0] = af_m[i][0][0][0]

y = np.zeros([y_m.size[0], 1])
for i in range(y_m.size[0]):
        y[i,0] = y_m[i][0][0][0]

y = y.ravel()

X = np.stack((ar, af), axis=-1)

X = X.reshape(17, -1)


TypeError: 'int' object is not subscriptable